### Load libs

In [31]:
import pandas as pd
import numpy as np
import sklearn
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
import re
from scipy import spatial
import time
import multiprocessing

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load data

In [32]:
df1=pd.read_excel(r"datasets/extremism.xlsx")

df=pd.DataFrame()
df['id'] = df1['id']
df['text'] = df1['text']
df['class'] = df1['class']
df = df.dropna()

In [33]:
df = pd.read_json(r'C:\Users\Zero\Downloads\ria_1k.json', lines = True,)

In [34]:
df

,text,title
0,<p><strong>автор мария балябина <br /></strong...,украинская люстрация: жертвоприношение во власти
1,"<p><strong></strong></p>\n<p><strong>москва, 2...",цена на нефть марки brent поднялась выше $73 з...
2,"<p><strong></strong></p>\n<p><strong>бангкок, ...",число пострадавших в аварии в таиланде россиян...
3,"<p><strong></strong></p>\n<p><strong>москва, 2...",собянин открыл детсад в хорошево-мневниках в м...
4,"<p><strong></strong></p>\n<p><strong>москва, 2...","""ташир"" построит в москве перинатально-кардиол..."
...,...,...
995,"<p><strong></strong></p>\n<p><strong>москва, 1...",крупнейшими учениями для вс россии в 2015 году...
996,"<p><strong></strong></p>\n<p><strong>москва, 1...",рвсн россии проведут в 2015 году более ста учений
997,"<p><strong></strong></p>\n<p><strong>москва, 1...",сухопутные войска россии в 2015 году проведут ...
998,"<p><strong></strong></p>\n<p><strong>мехико, 3...",еще несколько сожженных тел нашли в мексиканск...


In [35]:
from nltk.tokenize import RegexpTokenizer
import time
t = time.time()


tokenizer = RegexpTokenizer(r'\w+')
df['clean'] = df['text'].astype('str') 
df.dtypes

df["tokens"] = df['text'].apply(tokenizer.tokenize)
# delete Stop Words

print('Time to tokenize everything: {} mins'.format(round((time.time() - t) / 60, 2)))
df.head()

Time to tokenize everything: 0.0 mins


,text,title,clean,tokens
0,<p><strong>автор мария балябина <br /></strong...,украинская люстрация: жертвоприношение во власти,<p><strong>автор мария балябина <br /></strong...,"[p, strong, автор, мария, балябина, br, strong..."
1,"<p><strong></strong></p>\n<p><strong>москва, 2...",цена на нефть марки brent поднялась выше $73 з...,"<p><strong></strong></p>\n<p><strong>москва, 2...","[p, strong, strong, p, p, strong, москва, 2, д..."
2,"<p><strong></strong></p>\n<p><strong>бангкок, ...",число пострадавших в аварии в таиланде россиян...,"<p><strong></strong></p>\n<p><strong>бангкок, ...","[p, strong, strong, p, p, strong, бангкок, 2, ..."
3,"<p><strong></strong></p>\n<p><strong>москва, 2...",собянин открыл детсад в хорошево-мневниках в м...,"<p><strong></strong></p>\n<p><strong>москва, 2...","[p, strong, strong, p, p, strong, москва, 2, д..."
4,"<p><strong></strong></p>\n<p><strong>москва, 2...","""ташир"" построит в москве перинатально-кардиол...","<p><strong></strong></p>\n<p><strong>москва, 2...","[p, strong, strong, p, p, strong, москва, 2, д..."


In [36]:
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('russian')
stopwords

stopwords = set(stopwords)

df['chistka_clean'] = df['clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df['chistka_text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Zero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
import re

patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

def lemmatize(doc):
    doc = re.sub(patterns, '', doc)
    tokens = []
    for token in doc.split():
        new_token=''
        for char in token:
            if (char not in 'abcdefghijklmnopqrstuvwxyz') and (char not in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
                new_token = str(new_token) + str(char)
        token=new_token
        if token:
            token = token.strip()
            tokens.append(token)
    return tokens

In [38]:
df['super_']=df['chistka_clean'].apply(lemmatize)

In [39]:
df['super_']=df['chistka_clean'].apply(lemmatize)

### Load models

In [40]:
w2v_model = Word2Vec(
    min_count=10,
    window=10,
    size=300,
    negative=10,
    alpha=0.5,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1,workers=4)

exp_model = w2v_model.wv.load_word2vec_format(r'models/exp_model.bin', binary=True)

In [41]:
exp_model.most_similar('образование')

[('высшее', 0.958590030670166),
 ('человеческое', 0.94144606590271),
 ('совместное', 0.9322394728660583),
 ('семейное', 0.9120088815689087),
 ('глупое', 0.9114097356796265),
 ('рубрика', 0.9108825325965881),
 ('разное', 0.9107043743133545),
 ('спокойствие', 0.906814694404602),
 ('общемто', 0.9057402610778809),
 ('обычное', 0.9044653177261353)]

## Prepare funcs

### Tokenization

In [42]:
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"

def comma2space(word):
    new_word = ''
    try:
        for char in word:
            if char in patterns:
                char = ' '
            new_word += char
    except TypeError:
        pass
    return new_word
    
def lemmatize(doc):
    doc = re.sub(patterns, '', doc)
    tokens = []
    for token in doc.split():
        if token:
            
            token = token.lower()
            token = token.strip()
            tokens.append(token)
    return tokens
def lower(doc):
    doc = re.sub(patterns, '', doc)
    tokens = []
    for token in doc.split():
        if token:
            
            token = token.lower()
            tokens.append(token)
            
    return tokens

def dropstopwords(docs):
    stopwords = nltk.corpus.stopwords.words('russian')
    sen =  []
    for doc in docs:
        if (doc not in stopwords):
            sen.append(doc)
    return sen

def dropcomma(doc):
    new_sen = ''
    for char in doc:
        if (char not in patterns):
            new_sen += char
    return new_sen

def tokenize(docs):
    docs = comma2space(docs)
    docs = lemmatize(docs)
    docs = dropstopwords(docs)
    return docs

### Medium word-vector

In [43]:
def mediumVector(sen):
    medVec = np.zeros(300).reshape((1, 300))
    count = 0
    for word in sen:
        try:
            vec = exp_model.word_vec(word)
            medVec += vec
            count += 1
        except:
            pass
    if count >= 0 :
        medVec = medVec / count
    return medVec

## Organise data 

In [44]:
data=df['text'].apply(tokenize)
data

0      [p, strong, автор, мария, балябина, br, strong...
1      [p, strong, strong, p, p, strong, москва, дек,...
2      [p, strong, strong, p, p, strong, бангкок, дек...
3      [p, strong, strong, p, p, strong, москва, дек,...
4      [p, strong, strong, p, p, strong, москва, дека...
                             ...                        
995    [p, strong, strong, p, p, strong, москва, дек,...
996    [p, strong, strong, p, p, strong, москва, дек,...
997    [p, strong, strong, p, p, strong, москва, дек,...
998    [p, strong, strong, p, p, strong, мехико, ноя,...
999    [p, strong, strong, p, p, strong, москва, ноя,...
Name: text, Length: 1000, dtype: object

In [45]:
df['tokenized']=df['text'].apply(tokenize)
df['medVec']=df['tokenized'].apply(mediumVector)
df

,text,title,clean,tokens,chistka_clean,chistka_text,super_,tokenized,medVec
0,<p><strong>автор мария балябина <br /></strong...,украинская люстрация: жертвоприношение во власти,<p><strong>автор мария балябина <br /></strong...,"[p, strong, автор, мария, балябина, br, strong...",<p><strong>автор мария балябина <br /></strong...,<p><strong>автор мария балябина <br /></strong...,"[автор, мария, балябина, украинская, люстрация...","[p, strong, автор, мария, балябина, br, strong...","[[-0.026847792075847435, -0.05230047442606191,..."
1,"<p><strong></strong></p>\n<p><strong>москва, 2...",цена на нефть марки brent поднялась выше $73 з...,"<p><strong></strong></p>\n<p><strong>москва, 2...","[p, strong, strong, p, p, strong, москва, 2, д...","<p><strong></strong></p> <p><strong>москва, 2 ...","<p><strong></strong></p> <p><strong>москва, 2 ...","[москва, дек, риа, новости, цена, нанефть, мар...","[p, strong, strong, p, p, strong, москва, дек,...","[[-0.026536655586419836, -0.05415772147373193,..."
2,"<p><strong></strong></p>\n<p><strong>бангкок, ...",число пострадавших в аварии в таиланде россиян...,"<p><strong></strong></p>\n<p><strong>бангкок, ...","[p, strong, strong, p, p, strong, бангкок, 2, ...","<p><strong></strong></p> <p><strong>бангкок, 2...","<p><strong></strong></p> <p><strong>бангкок, 2...","[бангкок, дек, риа, новости, число, россиян, п...","[p, strong, strong, p, p, strong, бангкок, дек...","[[-0.02384246904943891, -0.055898756041624253,..."
3,"<p><strong></strong></p>\n<p><strong>москва, 2...",собянин открыл детсад в хорошево-мневниках в м...,"<p><strong></strong></p>\n<p><strong>москва, 2...","[p, strong, strong, p, p, strong, москва, 2, д...","<p><strong></strong></p> <p><strong>москва, 2 ...","<p><strong></strong></p> <p><strong>москва, 2 ...","[москва, дек, риа, новости, строительство, дет...","[p, strong, strong, p, p, strong, москва, дек,...","[[-0.015161047186120413, -0.05694700340197111,..."
4,"<p><strong></strong></p>\n<p><strong>москва, 2...","""ташир"" построит в москве перинатально-кардиол...","<p><strong></strong></p>\n<p><strong>москва, 2...","[p, strong, strong, p, p, strong, москва, 2, д...","<p><strong></strong></p> <p><strong>москва, 2 ...","<p><strong></strong></p> <p><strong>москва, 2 ...","[москва, декабря, риа, новости, группа, компан...","[p, strong, strong, p, p, strong, москва, дека...","[[-0.026468425002151533, -0.06473511310827419,..."
...,...,...,...,...,...,...,...,...,...
995,"<p><strong></strong></p>\n<p><strong>москва, 1...",крупнейшими учениями для вс россии в 2015 году...,"<p><strong></strong></p>\n<p><strong>москва, 1...","[p, strong, strong, p, p, strong, москва, 1, д...","<p><strong></strong></p> <p><strong>москва, 1 ...","<p><strong></strong></p> <p><strong>москва, 1 ...","[москва, дек, риа, новости, главными, стратеги...","[p, strong, strong, p, p, strong, москва, дек,...","[[-0.02012034799368616, -0.05219490550696216, ..."
996,"<p><strong></strong></p>\n<p><strong>москва, 1...",рвсн россии проведут в 2015 году более ста учений,"<p><strong></strong></p>\n<p><strong>москва, 1...","[p, strong, strong, p, p, strong, москва, 1, д...","<p><strong></strong></p> <p><strong>москва, 1 ...","<p><strong></strong></p> <p><strong>москва, 1 ...","[москва, дек, риа, новости, ракетные, войска, ...","[p, strong, strong, p, p, strong, москва, дек,...","[[-0.023521798985222213, -0.056317618125691436..."
997,"<p><strong></strong></p>\n<p><strong>москва, 1...",сухопутные войска россии в 2015 году проведут ...,"<p><strong></strong></p>\n<p><strong>москва, 1...","[p, strong, strong, p, p, strong, москва, 1, д...","<p><strong></strong></p> <p><strong>москва, 1 ...","<p><strong></strong></p> <p><strong>москва, 1 ...","[москва, дек, риа, новости, сухопутные, войска...","[p, strong, strong, p, p, strong, москва, дек,...","[[-0.023370054186297916, -0.0684761208935808, ..."
998,"<p><strong></strong></p>\n<p><strong>мехико, 3...",еще несколько сожженных тел нашли в мексиканск...,"<p><strong></strong></p>\n<p><st

In [46]:
df['class'] = df['class'].astype('int32')

KeyError: 'class'

In [ ]:
df

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df['text'], df['class'], test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [11]:
from sklearn.pipeline import Pipeline
# pipeline позволяет объединить в один блок трансформер и модель, что упрощает написание кода и улучшает его читаемость
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer преобразует тексты в числовые вектора, отражающие важность использования каждого слова из некоторого набора слов (количество слов набора определяет размерность вектора) в каждом тексте
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
# линейный классификатор и классификатор методом ближайших соседей
from sklearn import metrics
# набор метрик для оценки качества модели
from sklearn.model_selection import GridSearchCV
# модуль поиска по сетке параметров

In [12]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
sgd_ppl_clf.fit(X_train, y_train)
knb_ppl_clf.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
pred = knb_ppl_clf.predict(['Русские вперед убивай нерусских', 'Сегодня хорошая погода', 'Винда', "В чем смысл жизни", "Режьте ебаных чурок", "Хочу свалить в Татарстан"])
print(pred)

In [24]:
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95        21
           1       0.80      0.80      0.80        15
           3       0.82      0.90      0.86        20

    accuracy                           0.88        56
   macro avg       0.87      0.87      0.87        56
weighted avg       0.88      0.88      0.88        56



In [25]:
predicted_sgd = knb_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       0.95      0.82      0.88        22
           1       0.93      0.58      0.72        24
           3       0.45      1.00      0.62        10

    accuracy                           0.75        56
   macro avg       0.78      0.80      0.74        56
weighted avg       0.85      0.75      0.76        56



In [26]:
parameters = { 
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              'tfidf__strip_accents':['ascii', 'unicode', None],
               'tfidf__ngram_range':[(1,2), (2,3), (3,4)]
              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train, y_train)
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.828327922077922 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'hinge', 'sgd_clf__penalty': 'elasticnet', 'tfidf__ngram_range': (1, 2), 'tfidf__strip_accents': None}


In [ ]:
model

In [27]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('sgd_clf', SGDClassifier(penalty='elasticnet'))])
sgd_ppl_clf.fit(X_train, y_train)
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       1.00      0.86      0.93        22
           1       0.87      0.68      0.76        19
           3       0.68      1.00      0.81        15

    accuracy                           0.84        56
   macro avg       0.85      0.85      0.83        56
weighted avg       0.87      0.84      0.84        56



In [28]:
from sklearn.ensemble import RandomForestClassifier
forest = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('forest_clf', RandomForestClassifier())])


In [29]:
forest.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                ('forest_clf', RandomForestClassifier())])

In [30]:
forest.predict(['Русский язык - самый могучий язык в мире', 'Слава нации, смерть врагам', 'Жизнь прекрасна', 'Белорусские власти объявили забастовку', 'Привет'])

array([3, 3, 3, 3, 3])

In [31]:
predicted_forest = forest.predict(X_test)
print(metrics.classification_report(predicted_forest, y_test))

              precision    recall  f1-score   support

           0       0.42      1.00      0.59         8
           1       0.53      0.67      0.59        12
           3       0.91      0.56      0.69        36

    accuracy                           0.64        56
   macro avg       0.62      0.74      0.62        56
weighted avg       0.76      0.64      0.65        56



In [32]:
forest.score(X_test, y_test)

0.6428571428571429

In [33]:
sgd_ppl_clf.score(X_test, y_test)

0.8392857142857143

In [34]:
knb_ppl_clf.score(X_test, y_test)

0.75

In [106]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svm", SVC())])
param_grid = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
              'svm__gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))

Best cross-validation accuracy: 0.65
Test set score: 0.66
Best parameters: {'svm__C': 10, 'svm__gamma': 0.1}


In [ ]:
pred=grid.predict(['выброшусь в окно'])
print(pred)

### пробуем всё

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("MB",  MLPClassifier(solver='adam'))])
param_grid = { "MB__hidden_layer_sizes":range(10,100,20),
            
              
              'MB__alpha': [0.0000001, 0.000001, 0000.1, 0.0001, 0.001, 0.01,0.1,1,10,100]}
grid = GridSearchCV(pipe, param_grid=param_grid)
grid.fit(X_train, y_train)
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))

C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer

C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer

C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer

C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer

C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer

C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer

C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer

Best cross-validation accuracy: 0.82
Test set score: 0.78
Best parameters: {'MB__alpha': 0.1, 'MB__hidden_layer_sizes': 90}


C:\Users\Zero\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [18]:
pred=grid.predict([''])
print(pred)

[3.]


In [16]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

In [17]:
pipe1 = Pipeline([('preprocessing', TfidfVectorizer()), ('classifier', SVC())])

In [18]:
param_grid = [
    {'classifier': [SVC()], 'preprocessing': [TfidfVectorizer()],
     'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
    {'classifier': [RandomForestClassifier(n_estimators=100)],
     'preprocessing': [TfidfVectorizer()], 'classifier__max_features': [1, 2, 3]},
{'classifier':[SGDClassifier()],'classifier__alpha': [0.0000001, 0.000001, 0000.1, 0.0001, 0.001, 0.01,0.1,1,10,100],
    'classifier__n_jobs': [-3,-2,-1,1,2,3]}]
grid1 = GridSearchCV(pipe1, param_grid, cv=5)
grid1.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

NameError: name 'RandomForestClassifier' is not defined

In [206]:
pred=grid.predict(['посоветуйте как покнчить жизнь самоубийством'])
print(pred)

[3]


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df['text'], df['class'], test_size=0.3, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

KeyError: 'class'

In [181]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

vect = CountVectorizer(min_df=3).fit(X_train1)
X_train1 = vect.transform(X_train1)
print("X_train:\n{}".format(repr(X_train1)))

X_train:
<151x282 sparse matrix of type '<class 'numpy.int64'>'
	with 1741 stored elements in Compressed Sparse Row format>


In [182]:
feature_names = vect.get_feature_names()
print("Количество признаков: {}".format(len(feature_names)))
print("Первые 20 признаков:\n{}".format(feature_names[:20]))
print("Признаки с 20010 по 20030:\n{}".format(feature_names[100:230]))
print("Каждый 2000-й признак:\n{}".format(feature_names[::20]))

Количество признаков: 282
Первые 20 признаков:
['2014', '31', '40', 'автор', 'активности', 'аллах', 'без', 'бей', 'более', 'боль', 'больше', 'будет', 'буду', 'будут', 'будь', 'бы', 'был', 'была', 'были', 'было']
Признаки с 20010 по 20030:
['ли', 'лишь', 'людей', 'люди', 'мария', 'между', 'меня', 'место', 'мид', 'мир', 'мире', 'мне', 'много', 'могут', 'может', 'можешь', 'москва', 'москве', 'моя', 'мы', 'мысли', 'на', 'надо', 'накануне', 'нам', 'нас', 'наших', 'не', 'негра', 'некоторые', 'ненавижу', 'нет', 'ни', 'никогда', 'никому', 'никто', 'ним', 'них', 'ничего', 'но', 'ну', 'об', 'обвиняют', 'общества', 'один', 'одно', 'одного', 'окружающей', 'он', 'она', 'они', 'осталось', 'от', 'ответ', 'отметила', 'отношения', 'официальный', 'очень', 'печи', 'плохо', 'по', 'под', 'пока', 'поколение', 'поколения', 'политика', 'политики', 'после', 'последнее', 'потом', 'потому', 'почему', 'поэтому', 'права', 'представитель', 'пресс', 'при', 'придется', 'принять', 'проблем', 'проблемы', 'просто', 'про

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
scores = cross_val_score(LogisticRegression(), X_train1, y_train1, cv=5)
print("Средняя правильность перекр проверки: {:.2f}".format(np.mean(scores)))

NameError: name 'X_train1' is not defined

In [30]:
pipe1 = Pipeline([('preprocessing', TfidfVectorizer()), ('classifier', SVC())])

In [14]:
param_grid = [
    {'classifier': [SVC()], 'preprocessing': [TfidfVectorizer()],
     'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
    {'classifier': [RandomForestClassifier(n_estimators=100)],
     'preprocessing': [TfidfVectorizer()], 'classifier__max_features': [1, 2, 3]},
{'classifier':[SGDClassifier()],'classifier__alpha': [0.0000001, 0.000001, 0000.1, 0.0001, 0.001, 0.01,0.1,1,10,100],
    'classifier__n_jobs': [-3,-2,-1,1,2,3]}
grid2 = GridSearchCV(pipe1, param_grid, cv=5)
grid2.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test1, y_test1)))

SyntaxError: invalid syntax (<ipython-input-14-78c89ae66140>, line 9)

In [ ]:
import joblib
filename = 'best_Clss.joblib.pkl'
_ = joblib.dump(grid1, filename, compress=9)

In [190]:
pred=grid2.predict(['Русские вперед убивай нерусских', 'Сегодня хорошая погода', 'Виндовс', "В чем смысл жизни", "Режьте ебаных чурок", "Хочу свалить в Татарстан"])
print(pred)

[3 1 3 1 3 1]


KeyError: 'preprocessing'

ValueError: Iterable over raw text documents expected, string object received.